In [1]:
from evoframe.reward_builders import RewardBuilderGame
from evoframe.population_update_builders import PopulationUpdateBuilderStatic
from evoframe.selector_function import SelectorFunctionFactory
from evoframe import PopulationManager
from evoframe.models import FeedForwardNetwork
from evoframe.models import ActivationFunctions
from evoframe.games import Game
from evoframe import get_agent_wrapper_func
import evoframe.func_with_context as fwc

import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from evoframe.experiment_results import plot_rewards, show_best_fnn_weights, get_best_model_of_epoch

%matplotlib inline

In [2]:
import collections

def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)

In [3]:
import imageio

def to_gif(experiment_name, num_epochs, duration=0.5):
    for i in range(1, num_epochs+1):
        print(i)
        f = show_best_fnn_weights(experiment_name, i)
        b = f.to_image(format="png")
        with open("images/image{}.png".format(i), "wb") as fi:
            fi.write(b)
    images = []
    filenames = ["images/image{}.png".format(i) for i in range(1, num_epochs+1)]
    for i,filename in enumerate(filenames):
        print(i)
        images.append(imageio.imread(filename))
    imageio.mimsave('nn.gif', images, duration=duration)

# Guess the point

In [4]:
from evoframe.games import GuessPoint

In [ ]:
# Define global context
context = recursively_default_dict()

# Game
game_creation_func = lambda context: GuessPoint(np.array([0.2,0.8,0.5]), np.array([0.4, 0.5, 10]))
game_creation_func = fwc.func_with_context(game_creation_func, context=context)

# Game-Model interface
predict_func = lambda model, inputs: model.predict(inputs)
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
reward_function = RewardBuilderGame() \
.with_game_creation_function(game_creation_func) \
.with_agent_wrapper_func(agent_wrapper_func) \
.with_context(context) \
.get()

# Model
layer_sizes = [3, 5, 3]
get_model_func = lambda context: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_relu(), ActivationFunctions.get_id())
get_model_func = fwc.func_with_context(get_model_func, context=context)

# Update population function
get_new_pop_f = PopulationUpdateBuilderStatic() \
.add_operator("es_1_mutation", 1, 0.3, 0.3) \
.add_operator("es_2_crossover", 0.01, 0.8) \
.add_operator("es_1_copy", 0.01) \
.add_operator("es_n_rewards_gradient", 0.01) \
.add_selector_f(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.with_context(context) \
.get()

# Population manager
pop_size = 500
pm = PopulationManager(pop_size, get_model_func, reward_function, get_new_pop_f, context)

# Run population manager
num_epochs = 10
experiment_name = "guesspoint"
pm.run(num_epochs, experiment_name)

In [ ]:
plot_rewards(experiment_name)

In [ ]:
to_gif(experiment_name, num_epochs)

![...](nn.gif "segment")

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

# Guess the function

In [ ]:
from evoframe.games import GuessFunction

In [ ]:
# Define global context
context = recursively_default_dict()

# Game
game_func = lambda i: np.array([2*i[0]-3*i[1]+4, i[1]-8*i[2]-5])
input_dim = 3
input_domains = [(-1,1),(-1,1),(3,7)]
sample_every = [0.1, 0.1, 0.3]
game_creation_function = lambda context: GuessFunction(game_func, input_dim, input_domains, sample_every)
game_creation_function = fwc.func_with_context(game_creation_function, context=context)

# Model
layer_sizes = [3, 5, 2]
get_model_func = lambda context: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_arctan(), ActivationFunctions.get_id())
get_model_func = fwc.func_with_context(get_model_func, context=context)

# Game-Model interface
predict_func = lambda model, inputs: model.predict(inputs)
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
reward_function = RewardBuilderGame() \
.with_game_creation_function(game_creation_function) \
.with_agent_wrapper_func(agent_wrapper_func) \
.with_context(context) \
.get()

# Update population function
get_new_pop_f = PopulationUpdateBuilderStatic() \
.add_operator("es_1_mutation", 0.2, 0.1, 0.1) \
.add_operator("es_1_mutation", 0.2, 0.1, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.3, 0.3) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.1) \
.add_operator("es_2_crossover", 0.1, 0.8) \
.add_operator("es_1_copy", 0.1) \
.add_operator("es_n_rewards_gradient", 0.1) \
.add_selector_f(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.with_context(context) \
.get()

# Population manager
pop_size = 50
pickle_models_after_gens = 1
pm = PopulationManager(pop_size, get_model_func, reward_function, get_new_pop_f, pickle_models_after_gens, context)

# Run population manager
num_epochs = 50
experiment_name = "guessfunction"
pm.run(num_epochs, experiment_name)

In [ ]:
plot_rewards(experiment_name)

In [ ]:
to_gif(experiment_name, num_epochs)

In [ ]:
from evoframe.experiment_results import get_best_model_of_epoch

In [ ]:
test_array = np.array([0.5,0.5,5])
get_best_model_of_epoch(experiment_name, num_epochs).predict(test_array), game_func(test_array)

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

# Play Tris, 9 inputs

In [ ]:
from evoframe.reward_builders.reward_builder_game import TournamentMode

In [ ]:
from evoframe.games import Tris

In [ ]:
##### Define global context
context = recursively_default_dict()

# Game
game_creation_function = lambda context: Tris()
game_creation_function = fwc.func_with_context(game_creation_function, context=context)

# Model
layer_sizes = [9, 18, 9]
get_model_func = lambda context: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_sigmoid(), ActivationFunctions.get_sigmoid())
get_model_func = fwc.func_with_context(get_model_func, context=context)

# Game-Model interface
def predict_func(model, inputs):
    # In Tris, 'inputs' is a 3x3 np.array, thus we must flatten it to a 9x1 np.array
    # output is a 9x1 vector, where the cell with highest value corresponding
    # to a valid move will be the chosen move
    return model.predict(inputs.flatten())[0]
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
# Since Tris is a two-players game, we can compute the reward with a tournament against the current generation
keep_only = 30
reward_function = RewardBuilderGame() \
.with_game_creation_function(game_creation_function) \
.with_agent_wrapper_func(agent_wrapper_func) \
.with_competitive_tournament(TournamentMode.VS_PEAKS) \
.with_keep_only(keep_only) \
.with_context(context) \
.get()

# Update population function
get_new_pop_f = PopulationUpdateBuilderStatic() \
.add_operator("es_1_mutation", 0.2, 0.1, 0.1) \
.add_operator("es_1_mutation", 0.2, 0.1, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.3, 0.3) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.1) \
.add_operator("es_2_crossover", 0.1, 0.8) \
.add_operator("es_1_copy", 0.1) \
.add_operator("es_n_rewards_gradient", 0.1, 0.05) \
.add_operator("es_n_rewards_gradient", 0.1, 0.1) \
.add_operator("es_n_rewards_gradient", 0.1, 0.3) \
.add_operator("es_n_rewards_gradient", 0.1, 0.5) \
.add_operator("es_n_rewards_gradient", 0.1, 1.0) \
.add_selector_f(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.with_context(context) \
.get()

# Population manager
pop_size = 100
pm = PopulationManager(pop_size, get_model_func, reward_function, get_new_pop_f, context)

# Run population manager
num_epochs = 1000
experiment_name = "tris"
#pm.run(num_epochs, experiment_name)

In [ ]:
experiment_name = "tris"
num_epochs = 1000

In [ ]:
plot_rewards(experiment_name, epochs=list(range(800,1000)))

In [ ]:
to_gif(experiment_name, num_epochs, duration=0.04)

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

In [ ]:
class AgentHuman:
    def __init__(self):
        pass
    
    def predict(self, board):
        move = int(input("Select move: "))
        prediction = [1 if i == move else 0 for i in range(9)]
        return prediction

In [ ]:
def predict_func(model, inputs):
    # In Tris, 'inputs' is a 3x3 np.array, thus we must flatten it to a 9x1 np.array
    # output is a 9x1 vector, where the cell with highest value corresponding
    # to a valid move will be the chosen move
    return model.predict(inputs.flatten())[0]
agent_wrapper_func = get_agent_wrapper_func(predict_func)

In [ ]:
game = Tris()
game.play(AgentHuman(), agent_wrapper_func(get_best_model_of_epoch(experiment_name, num_epochs)), interactive=True)

In [ ]:
import plotly.express as px

In [ ]:
def show_predictions(model, inp):
    out = agent_wrapper_func(get_best_model_of_epoch(experiment_name, num_epochs)).predict(inp)
    out = out.reshape((3,3))
    shape = out.shape
    data = [(row+1, col+1, out[row][col]) for row in range(shape[0]) for col in range(shape[1])]
    columns = ["neuron_input", "neuron_output", "value"]
    df = pd.DataFrame(data=data, columns=columns)
    return px.density_heatmap(df, x="neuron_output", y="neuron_input", z="value",
                                         histfunc="sum", color_continuous_scale="RdYlGn", range_color=(-2,2),
                                         nbinsx=shape[1], nbinsy=shape[0],
                                         range_x=(0.5, shape[1]+0.5), range_y=(0.5, shape[0]+0.5))

In [ ]:
def get_random_input():
    return np.array([[np.random.choice([-1,0,1]) for i in range(3)] for j in range(3)])

In [ ]:
model = agent_wrapper_func(get_best_model_of_epoch(experiment_name, 100))
inp = get_random_input()
show_predictions(model, inp)

# Play Tris, 18 inputs

In [ ]:
# Define global context
context = recursively_default_dict()

# Game
game_creation_function = lambda context: Tris()
game_creation_function = fwc.func_with_context(game_creation_function, context=context)

# Model
layer_sizes = [18, 9, 9]
get_model_func = lambda context: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_sigmoid(), ActivationFunctions.get_sigmoid())
get_model_func = fwc.func_with_context(get_model_func, context=context)

# Game-Model interface
def predict_func(model, inputs):
    # In Tris, 'inputs' is a 3x3 np.array
    # output is a 9x1 vector, where the cell with highest value corresponding
    # to a valid move will be the chosen move
    board = inputs
    inputs = []
    for row in board:
        for cell in row:
            if cell == Tris.PLAYER_1:
                inputs += [1, 0]
            elif cell == Tris.PLAYER_2:
                inputs += [0, 1]
            else:
                inputs += [0, 0]
    return model.predict(np.array(inputs))[0]
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
# Since Tris is a two-players game, we can compute the reward with a tournament against the current generation
keep_only = 30
reward_function = RewardBuilderGame() \
.with_game_creation_function(game_creation_function) \
.with_agent_wrapper_func(agent_wrapper_func) \
.with_competitive_tournament(TournamentMode.VS_BEST_OF_EACH_GEN) \
.with_keep_only(keep_only) \
.with_context(context) \
.get()

# Update population function
get_new_pop_f = PopulationUpdateBuilderStatic() \
.add_operator("es_1_mutation", 0.2, 0.1, 0.1) \
.add_operator("es_1_mutation", 0.2, 0.1, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.3, 0.3) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.1) \
.add_operator("es_2_crossover", 0.1, 0.8) \
.add_operator("es_1_copy", 0.1) \
.add_operator("es_n_rewards_gradient", 0.1) \
.add_selector_f(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.with_context(context) \
.get()

# Population manager
pop_size = 30
pm = PopulationManager(pop_size, get_model_func, reward_function, get_new_pop_f, keep_only, context)

# Run population manager
num_epochs = 300
experiment_name = "tris_18"
pm.run(num_epochs, experiment_name)

In [ ]:
plot_rewards(experiment_name)

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

In [ ]:
to_gif(experiment_name, num_epochs, duration=0.04)

In [ ]:
class AgentHuman:
    def __init__(self):
        pass
    
    def predict(self, board):
        move = int(input("Select move: "))
        prediction = [1 if i == move else 0 for i in range(9)]
        return prediction

In [ ]:
game = Tris()
game.play(agent_wrapper_func(get_best_model_of_epoch(experiment_name, num_epochs)), AgentHuman(), interactive=True)

# Monkeywars

In [ ]:
import numpy as np
from evoframe.games import Game

class Tris(Game):
    """Player1 starts. Rewards of both players are returned."""
    PLAYER_1 = 1
    PLAYER_2 = -1
    EMPTY = 0
    DRAW = 0
    CONTINUE = 2
    PLAYERS = [PLAYER_1, PLAYER_2]

    def __init__(self):
        self.board = np.array([np.array([self.EMPTY for i in range(3)]) for j in range(3)])

    def check_win(self):
        # check rows
        board = self.board
        for row in board:
            for player in self.PLAYERS:
                if np.all(np.equal(row, np.full(3, player))):
                    return player

        # check cols
        board = self.board.transpose()
        for row in board:
            for player in self.PLAYERS:
                if np.all(np.equal(row, np.full(3, player))):
                    return player

        # check diagonals
        diags = []
        diags.append(np.array([board[i][i] for i in range(3)]))
        diags.append(np.array([board[i][3 - i - 1] for i in range(3)]))
        for row in diags:
            for player in self.PLAYERS:
                if np.all(np.equal(row, np.full(3, player))):
                    return player

        # check draw
        exist_empty = False
        for row in self.board:
            for cell in row:
                if cell == self.EMPTY:
                    exist_empty = True
        if not exist_empty:
            return self.DRAW

        return self.CONTINUE

    def extract_move(self, prediction):
        highest_value = -100000
        highest_value_index = -1
        for i,pred in enumerate(prediction):
            if self.board[i//3][i%3] == self.EMPTY and pred > highest_value:
                highest_value = pred
                highest_value_index = i
        return highest_value_index

    def do_move(self, move, player):
        self.board[move//3][move%3] = player

    def opposite_board(self):
        return np.array([np.array([self.PLAYER_2 if self.board[row][col] == self.PLAYER_1
                                   else self.PLAYER_1 if self.board[row][col] == self.PLAYER_2
                                   else self.EMPTY for col in range(3)]) for row in range(3)])

    def play(self, agent_1, agent_2, interactive=False):
        player_turn = self.PLAYER_1

        if interactive:
            self.print_board()

        result = self.check_win()
        while result == self.CONTINUE:
            if player_turn == self.PLAYER_1:
                prediction = agent_1.predict(self.board)
            else:
                prediction = agent_2.predict(self.opposite_board())

            move = self.extract_move(prediction)
            self.do_move(move, player_turn)

            if interactive:
                self.print_board()

            if player_turn == self.PLAYER_1:
                player_turn = self.PLAYER_2
            else:
                player_turn = self.PLAYER_1

            result = self.check_win()

        if interactive:
                self.print_board()

        opponent_result = self.PLAYER_2 if result == self.PLAYER_1 else self.PLAYER_1 if result == self.PLAYER_2 else self.DRAW
        return result, opponent_result

    def print_board(self):
        for row in self.board:
            for cell in row:
                print(cell, end=" ")
            print("")
        print("-"*30)
